In [219]:
import sys
import os
import re
from collections import Counter
import pandas as pd
from Bio import SeqIO
from pyBioInfo.IO.File import GtfFile, GtfTranscriptBuilder, GtfGeneBuilder, FastaFile

In [3]:
with GtfFile("results/assembly/final.gtf.gz") as f:
    records = [x for x in f]
print(len(records))

4316589


In [4]:
genes = list(GtfGeneBuilder(records))
print(len(genes))

47413


In [8]:
genes = list(filter(lambda item: item.chrom != "NC_016870.1", genes))
print(len(genes))

47376


In [10]:
transcripts = []
for gene in genes:
    for transcript in gene.transcripts:
        transcripts.append(transcript)
print(len(transcripts))

166139


In [80]:
cpat = pd.read_csv("results/function/cpat/predict.ncbi.tsv", sep="\t", index_col=0)  
print(len(cpat))

In [20]:
mapper1 = dict()
for tid, group in cpat.groupby(by="Transcript"):
    mapper1[tid] = group

In [36]:
for gene in genes:
    for transcript in gene.transcripts:
        transcript.cpat = mapper1.get(transcript.name)
    array = [t.cpat for t in gene.transcripts]
    array = list(filter(lambda item: item is not None, array))
    if len(array) > 0:
        gene.cpat = pd.concat(array)
    else:
        gene.cpat = None

In [74]:
diamond = pd.read_csv("results/function/diamond/nr.modified.annotated.tsv", sep="\t")
diamond["Transcript"] = [x.split(":")[0] for x in diamond["ID"]]

In [78]:
mapper2 = dict()
for tid, group in diamond.groupby(by="Transcript"):
    mapper2[tid] = group

In [79]:
for gene in genes:
    for transcript in gene.transcripts:
        transcript.diamond = mapper2.get(transcript.name)
    array = [t.diamond for t in gene.transcripts]
    array = list(filter(lambda item: item is not None, array))
    if len(array) > 0:
        gene.diamond = pd.concat(array)
    else:
        gene.diamond = None

In [161]:
d1 = pd.read_csv("results/function/interpro/pfam.modified.tsv", sep="\t")
d2 = pd.read_csv("results/function/interpro/PANTHER.modified.tsv", sep="\t")
# interpro = pd.concat([d1, d2])
interpro = d1
interpro["Transcript"] = [x.split(":")[0] for x in interpro["Accession"]]

In [ ]:
mapper3 = dict()
for tid, group in interpro.groupby(by="Transcript"):
    mapper3[tid] = group

In [ ]:
for gene in genes:
    for transcript in gene.transcripts:
        transcript.interpro = mapper3.get(transcript.name)
    array = [t.interpro for t in gene.transcripts]
    array = list(filter(lambda item: item is not None, array))
    if len(array) > 0:
        gene.interpro = pd.concat(array)
    else:
        gene.interpro = None

In [152]:
rows = []
for gene in genes:
    if gene.diamond is not None:
        tmp = gene.diamond
        tmp = tmp[(tmp["Evalue"] < 1e-6) & (tmp["PercIdent"] >= 80)]
        tmp = tmp.sort_values(by="BitScore", ascending=False)
        if len(tmp) > 0:
            row = tmp.iloc[0]
            row.name = gene.name
            rows.append(row)
diamond_result = pd.concat(rows, axis=1).T
diamond_result.index.name = "GeneID"

In [180]:
diamond_result.to_csv("results/function/analysis/diamond_result.tsv", sep="\t")

In [171]:
rows = []
for gene in genes:
    if gene.interpro is not None:
        tmp = gene.interpro
        tmp = tmp[(tmp["Evalue"] < 1e-6) & (tmp["GO"] != "-")]
        tmp = tmp.sort_values(by="Evalue", ascending=True)
        if len(tmp) > 0:
            row = tmp.iloc[0]
            row.name = gene.name
            rows.append(row)
interpro_result = pd.concat(rows, axis=1).T
interpro_result.index.name = "GeneID"

In [178]:
interpro_result.to_csv("results/function/analysis/interpro_result.tsv", sep="\t")

In [218]:
lines = []
for gene in genes:
    for transcript in gene.transcripts:
        exon_list = transcript.blocks
        cds_list = None
        if transcript.cpat is not None:
            tmp = transcript.cpat
            tmp = tmp[tmp["Coding_Prob"] >= 0.5]
            if len(tmp) > 0:
                tmp = transcript.cpat.sort_values(by="Coding_Prob", ascending=False)
                row = tmp.iloc[0]
                tid, i1, i2 = row.name.split(":")
                i1, i2 = int(i1), int(i2) - 1
                p1, p2 = transcript.position(i1), transcript.position(i2)
                p1, p2 = min(p1, p2), max(p1, p2) + 1
                obj = transcript.clip(p1, p2)
                cds_list = obj.blocks
        attri =  "gene_id \"%s\"; transcript_id \"%s\";" % (gene.name, transcript.name)
        # transcript
        line = [transcript.chrom, "Assembly", "transcript", 
                transcript.start + 1, transcript.end, 
                ".", transcript.strand, ".", attri]
        lines.append(line)
        # exon
        for start, end in exon_list:
            line = [transcript.chrom, "Assembly", "exon", 
                    start + 1, end, ".", transcript.strand, ".", attri]
            lines.append(line)
        # CDS
        if cds_list:
            offset = 0
            for start, end in cds_list:
                frame = 0
                if offset % 3 == 1:
                    frame = 2
                if offset % 3 == 2:
                    frame = 1
                line = [transcript.chrom, "Assembly", "CDS", 
                    start + 1, end, ".", transcript.strand, frame, attri]
                lines.append(line)
                offset += (end - start)
    # break

In [220]:
with open("results/function/analysis/final.cds.gtf", "w+") as fw:
    for line in lines:
        fw.write("\t".join(map(str, line)) + "\n")
cmd = "bedtools sort -i results/function/analysis/final.cds.gtf | bgzip -f -c > results/function/analysis/final.cds.gtf.gz"
assert os.system(cmd) == 0
cmd = "tabix -f -p gff results/function/analysis/final.cds.gtf.gz"
assert os.system(cmd) == 0